In [56]:
import numpy as np
import pandas as pd

In [57]:
import yaml

with open("../security.yaml") as f:
    SYS_CONFIG = yaml.safe_load(f)

# `SubwayBiStation-IntervalTime.ipynb`
> API 정보 - https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15001019#tab_layer_detail_function\
> `소모시간` = 도시철도가 어떤 역에서 다른 역까지 이동하는데 걸리는 순수 시간\
> `정차시간` = 도시철도가 어떤 역에서 도착한 후 정지하여 다음 역으로 출발하기 전 까지의 순수 시간\
> `실질배차간격` = `철도사용자의최악대기시간(환승시간제외)` = 철도사용자가 한 역에서 어떤 열차가 지나가는 것을 보고 다음 열차가 올 때까지의 시간
- 두 역에서의 `소모시간`을 데이터
- 한 역에서의 `정차시간`을 데이터

## 1. `소모시간`, `정차시간` 을 알기 위해 API로 데이터 Loading

```py
ENCODING        "EUC-KR"
startSn         출발역명:str
startSc	        출발역코드:int      (ex.신평:101, 안평:414)
endSn	        도착역명:str
endSc	        도착역코드:int      (ex.신평:101, 안평:414)
dist	        이동거리:int        단위:0.1 km (ex 16->1.6 km)
time	        이동시간:int        단위:초(sec)
stoppingTime	정차시간:int        단위:초(sec)
exchange        환승구분:str|Null   "Y" => 환승역
                                   "N" => 경전철
                                   ""(공란) => 일반역```


In [58]:
# GET API REQ
url = "http://data.humetro.busan.kr/voc/api/open_api_distance.tnn"
params = {"serviceKey": SYS_CONFIG["API_KEY"], "act": "json", "numOfRows": 328}

import requests

response = requests.get(url, params=params)

In [69]:
# TRIM AND SAVE REQ
import json


# SET JSON DECODER
# https://stackoverflow.com/questions/45068797/how-to-convert-string-int-json-into-real-int-with-json-loads
class Decoder(json.JSONDecoder):
    def decode(self, s):
        result = super().decode(
            s
        )  # result = super(Decoder, self).decode(s) for Python 2.x
        return self._decode(result)

    def _decode(self, o):
        if isinstance(o, str):
            if o == "Y":
                return True
            elif o == "N":
                return False
            elif o == "":
                return None

            try:
                return int(o)
            except ValueError:
                return o
        elif isinstance(o, dict):
            return {k: self._decode(v) for k, v in o.items()}
        elif isinstance(o, list):
            return [self._decode(v) for v in o]
        else:
            return o

In [70]:
# TRIM AND SAVE REQ
with open(
    "../trimmed_data/SubwayTwoStation-IntervalTime.json", mode="w", encoding="UTF-8"
) as f:
    """UTF-8로 저장함."""
    INTERVAL_TIME_RAW = json.loads(
        req := response.content.decode("EUC-KR"), cls=Decoder
    )
    INTERVAL_TIME_RAW = INTERVAL_TIME_RAW["response"]["body"]["item"]
    json.dump(INTERVAL_TIME_RAW, f, indent=4)

In [77]:
#LOAD JSON FILE
with open("../trimmed_data/SubwayTwoStation-IntervalTime.json",mode="r",encoding="UTF-8") as f:
    TIME_INTERVAL =json.load(f)
print(TIME_INTERVAL[0])

{'dist': 17, 'endSc': 101, 'endSn': '신평', 'exchange': False, 'startSc': 100, 'startSn': '동매', 'stoppingTime': 30, 'time': 180}
